In [ ]:
%%capture
!pip install mne
!wget https://md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com/fshy54ypyh-1.zip -O data.zip
!unzip data.zip


In [ ]:
from glob import glob
import scipy.io
import torch.nn as nn
import torch
import numpy as np
import mne

In [ ]:
IDD_data_path='/content/Data/CleanData/CleanData_TDC/Rest'
TDC_data_path='/content/Data/Data/CleanData/Data/Data/CleanData/CleanData_IDD/Rest'
!rm '/content/Data/Data/CleanData/Data/Data/CleanData/CleanData_IDD/Rest/NDS001_Rest_CD(1).mat'

In [ ]:
def convertmat2mne(data):
  ch_names = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4']
  ch_types = ['eeg'] * 14
  sampling_freq=128
  info = mne.create_info(ch_names, ch_types=ch_types, sfreq=sampling_freq)
  info.set_montage('standard_1020')
  data=mne.io.RawArray(data, info)
  data.set_eeg_reference()
  data.filter(l_freq=1,h_freq=30)
  epochs=mne.make_fixed_length_epochs(data,duration=4,overlap=0)
  return epochs.get_data()

In [ ]:
%%capture
idd_subject=[]
for idd in glob(IDD_data_path+'/*.mat'):
  data=scipy.io.loadmat(idd)['clean_data']
  data=convertmat2mne(data)
  idd_subject.append(data)

In [ ]:
%%capture
tdc_subject=[]
for tdc in glob(TDC_data_path+'/*.mat'):
  data=scipy.io.loadmat(tdc)['clean_data']
  data=convertmat2mne(data)
  tdc_subject.append(data)
  

In [ ]:
control_epochs_labels=[len(i)*[0] for i in tdc_subject]
patients_epochs_labels=[len(i)*[1] for i in idd_subject]
print(len(control_epochs_labels),len(patients_epochs_labels))

7 7


In [ ]:
data_list=tdc_subject+idd_subject
label_list=control_epochs_labels+patients_epochs_labels
groups_list=[[i]*len(j) for i, j in enumerate(data_list)]
print(len(data_list),len(label_list),len(groups_list))


14 14 14


In [ ]:
from sklearn.model_selection import GroupKFold,LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
gkf=GroupKFold()
from sklearn.base import TransformerMixin,BaseEstimator
from sklearn.preprocessing import StandardScaler
#https://stackoverflow.com/questions/50125844/how-to-standard-scale-a-3d-matrix
class StandardScaler3D(BaseEstimator,TransformerMixin):
    #batch, sequence, channels
    def __init__(self):
        self.scaler = StandardScaler()

    def fit(self,X,y=None):
        self.scaler.fit(X.reshape(-1, X.shape[2]))
        return self

    def transform(self,X):
        return self.scaler.transform(X.reshape( -1,X.shape[2])).reshape(X.shape)

In [ ]:
import numpy as np
data_array=np.concatenate(data_list)
label_array=np.concatenate(label_list)
group_array=np.concatenate(groups_list)
data_array=np.moveaxis(data_array,1,2)

print(data_array.shape,label_array.shape,group_array.shape)

(420, 512, 14) (420,) (420,)


In [ ]:
accuracy=[]
for train_index, val_index in gkf.split(data_array, label_array, groups=group_array):
    train_features,train_labels=data_array[train_index],label_array[train_index]
    val_features,val_labels=data_array[val_index],label_array[val_index]
    scaler=StandardScaler3D()
    train_features=scaler.fit_transform(train_features)
    val_features=scaler.transform(val_features)
   

    break

In [ ]:
train_features.shape

(330, 512, 14)

In [ ]:
from tensorflow.keras.layers import Input,Dense,concatenate,Flatten,GRU,Conv1D
from tensorflow.keras.models import Model
#resource:https://github.com/dll-ncai/eeg_pre-diagnostic_screening/blob/master/code/chrononet/chrono.py

In [ ]:
def block(input):
  conv1 = Conv1D(32, 2, strides=2,activation='relu',padding="same")(input)
  conv2 = Conv1D(32, 4, strides=2,activation='relu',padding="causal")(input)
  conv3 = Conv1D(32, 8, strides=2,activation='relu',padding="causal")(input)
  x = concatenate([conv1,conv2,conv3],axis=2)
  return x

In [ ]:
input= Input(shape=(512,14))
block1=block(input)
block2=block(block1)
block3=block(block2)

In [ ]:
gru_out1 = GRU(32,activation='tanh',return_sequences=True)(block3)
gru_out2 = GRU(32,activation='tanh',return_sequences=True)(gru_out1)
gru_out = concatenate([gru_out1,gru_out2],axis=2)
gru_out3 = GRU(32,activation='tanh',return_sequences=True)(gru_out)
gru_out = concatenate([gru_out1,gru_out2,gru_out3])
gru_out4 = GRU(32,activation='tanh')(gru_out)


In [ ]:
predictions = Dense(1,activation='sigmoid')(gru_out4)
model = Model(inputs=input, outputs=predictions)

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_features,train_labels,epochs=10,batch_size=128,validation_data=(val_features,val_labels))

Epoch 1/10
3/3 [==============================] - 37s 852ms/step - loss: 0.7080 - accuracy: 0.4909 - val_loss: 0.6531 - val_accuracy: 0.7556
Epoch 2/10
3/3 [==============================] - 0s 44ms/step - loss: 0.6204 - accuracy: 0.6545 - val_loss: 0.5819 - val_accuracy: 0.9778
Epoch 3/10
3/3 [==============================] - 0s 41ms/step - loss: 0.5300 - accuracy: 0.8273 - val_loss: 0.5714 - val_accuracy: 0.6333
Epoch 4/10
3/3 [==============================] - 0s 46ms/step - loss: 0.3850 - accuracy: 0.9182 - val_loss: 0.3057 - val_accuracy: 1.0000
Epoch 5/10
3/3 [==============================] - 0s 42ms/step - loss: 0.2384 - accuracy: 0.9818 - val_loss: 0.2004 - val_accuracy: 0.9778
Epoch 6/10
3/3 [==============================] - 0s 38ms/step - loss: 0.1165 - accuracy: 0.9758 - val_loss: 0.0533 - val_accuracy: 1.0000
Epoch 7/10
3/3 [==============================] - 0s 42ms/step - loss: 0.0524 - accuracy: 0.9970 - val_loss: 0.0221 - val_accuracy: 1.0000
Epoch 8/10
3/3 [=========